# 準備

## artのインストール

In [ ]:
!pip3 install adversarial-robustness-toolbox

## ライブラリのインポート

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt

# TensorFlow with Keras.
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Conv2D, Lambda
from keras.layers import MaxPooling2D, GlobalAveragePooling2D, Dropout
tf.compat.v1.disable_eager_execution()
from keras import backend as K

# ART
import art
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import KerasClassifier

## CIFAR10のロード・前処理

In [ ]:
# CIFAR10のロード。
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

# CIFAR10のラベル。
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)

In [ ]:
# 正規化。
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# ラベルをOne-hot-vector化。
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Classifier(k-WTAなし)の作成



## モデル定義

In [ ]:
# モデルの定義。
inputs = Input(shape=(32, 32, 3))
x = Conv2D(64, (3, 3), padding='SAME', activation='relu')(inputs)
x = Conv2D(64, (3, 3), padding='SAME', activation='relu')(x)
x = Dropout(0.25)(x)
x = MaxPooling2D()(x)

x = Conv2D(128, (3,3), padding='SAME', activation='relu')(x)
x = Conv2D(128, (3,3), padding='SAME', activation='relu')(x)
x = Dropout(0.25)(x)
x = MaxPooling2D()(x)

x = Conv2D(256, (3,3), padding='SAME', activation='relu')(x)
x = Conv2D(256, (3,3), padding='SAME', activation='relu')(x)
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)
x = Dropout(0.25)(x)
y = Dense(10, activation='softmax')(x)

model = Model(inputs, y)

# モデルのコンパイル。
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_31 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_116 (Conv2D)          (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_117 (Conv2D)          (None, 32, 32, 64)        36928     
_________________________________________________________________
dropout_81 (Dropout)         (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_118 (Conv2D)          (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_119 (Conv2D)          (None, 16, 16, 128)       147

## 学習の実行

## モデルの精度評価

In [ ]:
# 学習の実行。
model.fit(X_train, y_train,
          batch_size=512,
          epochs=30,
          validation_data=(X_test, y_test),
          shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/30


KeyboardInterrupt: 

In [ ]:
model.evaluate(X_test, y_test)

# Classifier(k-WTAあり)の作成

## k-WTAの作成

In [ ]:
class KWTA(tf.keras.layers.Layer):
  def __init__(self, k=None, **kwargs) :
    super().__init__(**kwargs)
    self.k = k

  def call(self, x):
    n = x[0].shape[0] * x[0].shape[1] * x[0].shape[2]
    if self.k == None:
            k = n * 7 // 10
        else:
            k = self.k
    topk = tf.math.top_k(x[0], k=k)   # 上位k個の値を抽出
    topk_min = K.min(topk.values)         # topkの最小値を抽出
    comp = tf.dtypes.cast(x >= topk_min, tf.float32)   # topkの最小値以上の値の部分を1, より小さい値の部分を0にしたTensorを生成
    return tf.math.multiply(x,comp)        # x * compより、topkの最小値以上の値のみそのまま、他は0になるようにする

class KWTA2D(tf.keras.layers.Layer):
    def __init__(self, k=None, **kwargs):
        super().__init__(**kwargs)
        self.k = k
        
    def call(self, x):
        n = x[0].shape[0] * x[0].shape[1] * x[0].shape[2]
        if self.k == None:
            k = n * 7 // 10
        else:
            k = self.k
        x_flatten = tf.reshape(x[0],  [n])
        topk = tf.math.top_k(x_flatten, k=k)
        topk_min = K.min(topk.values) 
        comp = tf.dtypes.cast(x >= topk_min, tf.float32)
        return tf.math.multiply(x,comp)

## モデル定義

In [ ]:
# モデルの定義。
inputs = Input(shape=(32, 32, 3))
x = Conv2D(64, (3, 3), padding='SAME')(inputs)
x = KWTA2D()(x)
x = Conv2D(64, (3, 3), padding='SAME')(x)
x = KWTA2D()(x)
x = Dropout(0.25)(x)
x = MaxPooling2D()(x)

x = Conv2D(128, (3,3), padding='SAME')(x)
x = KWTA2D()(x)
x = Conv2D(128, (3,3), padding='SAME')(x)
x = KWTA2D()(x)
x = Dropout(0.25)(x)
x = MaxPooling2D()(x)

x = Conv2D(256, (3,3), padding='SAME')(x)
x = KWTA2D()(x)
x = Conv2D(256, (3,3), padding='SAME')(x)
x = KWTA2D()(x)
x = GlobalAveragePooling2D()(x)

x = Dense(1024)(x)
x = KWTA(512)(x)
x = Dropout(0.25)(x)
y = Dense(10, activation='softmax')(x)

model_kwta = Model(inputs, y)

# モデルのコンパイル。
model_kwta.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model_kwta.summary()

Model: "model_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_35 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_140 (Conv2D)          (None, 32, 32, 64)        1792      
_________________________________________________________________
kwt_a2d_56 (KWTA2D)          (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_141 (Conv2D)          (None, 32, 32, 64)        36928     
_________________________________________________________________
kwt_a2d_57 (KWTA2D)          (None, 32, 32, 64)        0         
_________________________________________________________________
dropout_93 (Dropout)         (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 16, 16, 64)        0  

## 学習の実行

In [ ]:
# 学習の実行。
model_kwta.fit(X_train, y_train,
          batch_size=512,
          epochs=30,
          validation_data=(X_test, y_test),
          shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 250s 5ms/sample - loss: 2.0748 - accuracy: 0.2090 - val_loss: 2.6120 - val_accuracy: 0.1786
Epoch 2/30
50000/50000 [==============================] - 246s 5ms/sample - loss: 1.7651 - accuracy: 0.3173 - val_loss: 1.9447 - val_accuracy: 0.3140
Epoch 3/30
50000/50000 [==============================] - 245s 5ms/sample - loss: 1.6901 - accuracy: 0.3650 - val_loss: 1.6827 - val_accuracy: 0.3882
Epoch 4/30
50000/50000 [==============================] - 230s 5ms/sample - loss: 1.4517 - accuracy: 0.4656 - val_loss: 1.4615 - val_accuracy: 0.4695
Epoch 5/30
50000/50000 [==============================] - 228s 5ms/sample - loss: 1.3163 - accuracy: 0.5188 - val_loss: 1.3549 - val_accuracy: 0.5228
Epoch 6/30
50000/50000 [==============================] - 227s 5ms/sample - loss: 1.2658 - accuracy: 0.5407 - val_loss: 1.3671 - val_accuracy: 0.5537
Epoch 7/30
50000/50000 [==========================

## モデルの精度評価

In [ ]:
model_kwta.evaluate(X_test, y_test)

[0.6477729416370391, 0.7924]

# 敵対的攻撃

## 非標的型攻撃としてのFGSMによる摂動を作成
以下、この摂動を上乗せした画像のことを「敵対的画像」と呼ぶ。

In [ ]:
# 入力データの特徴量の最小値・最大値を指定。
# 特徴量は0.0～1.0の範囲に収まるように正規化しているため、最小値は0.0、最大値は1.0とする。
min_pixel_value = 0.0
max_pixel_value = 1.0

### k-WTAなし

In [ ]:
# モデルをART Keras Classifierでラップ。
classifier = KerasClassifier(model=model, clip_values=(min_pixel_value, max_pixel_value), use_logits=False)
# FGSMインスタンスの作成。
attack = FastGradientMethod(estimator=classifier, eps=0.10, targeted=False)
# 敵対的サンプルの生成（ベース画像はテストデータとする）。
X_adv = attack.generate(x=X_test)

### k-WTAあり

In [ ]:
classifier_kwta = KerasClassifier(model=model_kwta, clip_values=(min_pixel_value, max_pixel_value), use_logits=False)
attack_kwta = FastGradientMethod(estimator=classifier_kwta, eps=0.10, targeted=False)
X_adv_kwta = attack_kwta.generate(X_test)

## 敵対的画像に対するモデルの精度評価

### k-WTAなし

In [ ]:
model.evaluate(X_adv, y_test)

### k-WTAあり

In [ ]:
model_kwta.evaluate(X_adv_kwta, y_test)

[6.555474308013916, 0.1146]